# SOCaaS Demo Attack

## A bit on the Stack Network
![stack network](img/StackNetwork.png)

URLs:
- https://jupyter-uow.1ez.xyz
- https://kibana-uow.1ez.xyz
- https://elasticsearch-uow.1ez.xyz
- https://praeco-uow.1ez.xyz
- https://blog-uow.1ez.xyz/blog/

### Attacker capability [9]
- Jupyter Notebook
```
Kali Linux
RAM Max 8gb
```
- Tools
```
nmap
unicornscan 
nikto
hping3
xprobe2
sqlmap
wpscan
iputils-ping 
metasploit-framework
```

### Defender capability [10]
- Filebeat Node
```
Ubuntu 20.04
```
- Tools
```
Filebeat
MySQL
Wordpress
Tshark (~ wireshark)
Suricata with DDOS rules
```

### Centralized Logging Management (or SOC) [2 -5]
- Elasticsearch (2 Nodes)
> Max JVM 4 gb ram /nodes
- Kibana (1 Nodes)
- Logstash (1 Nodes)

### First attack ~ NMAP
This attack would be reflected on tshark

In [1]:
%%bash
sudo nmap -O doblog-uow.1ez.xyz

Starting Nmap 7.91 ( https://nmap.org ) at 2021-03-01 09:03 UTC
Nmap scan report for doblog-uow.1ez.xyz (104.248.152.201)
Host is up (0.19s latency).
Not shown: 998 closed ports
PORT   STATE SERVICE
22/tcp open  ssh
80/tcp open  http
Aggressive OS guesses: Linux 5.0 - 5.4 (99%), Linux 5.4 (98%), Linux 4.15 - 5.6 (97%), Linux 5.1 (96%), Linux 2.6.32 - 3.13 (96%), Linux 5.0 - 5.3 (95%), Linux 2.6.22 - 2.6.36 (95%), Linux 3.10 - 4.11 (95%), Linux 5.0 (94%), Linux 3.10 (94%)
No exact OS matches for host (test conditions non-ideal).
Network Distance: 20 hops

OS detection performed. Please report any incorrect results at https://nmap.org/submit/ .
Nmap done: 1 IP address (1 host up) scanned in 8.44 seconds


## Second Attack ~ hping3
This attack would be reflected on suricata with it ddos rule. (+) Tshark

In [2]:
%%bash 
ping -c 4 doblog-uow.1ez.xyz

PING doblog-uow.1ez.xyz (104.248.152.201) 56(84) bytes of data.
64 bytes from 104.248.152.201 (104.248.152.201): icmp_seq=1 ttl=47 time=195 ms
64 bytes from 104.248.152.201 (104.248.152.201): icmp_seq=2 ttl=47 time=195 ms
64 bytes from 104.248.152.201 (104.248.152.201): icmp_seq=3 ttl=47 time=195 ms
64 bytes from 104.248.152.201 (104.248.152.201): icmp_seq=4 ttl=47 time=196 ms

--- doblog-uow.1ez.xyz ping statistics ---
4 packets transmitted, 4 received, 0% packet loss, time 3003ms
rtt min/avg/max/mdev = 194.869/195.135/195.590/0.288 ms


In [3]:
%%bash
sudo hping3 -c 1000 -S -p 80 doblog-uow.1ez.xyz

HPING doblog-uow.1ez.xyz (eth1 104.248.152.201): S set, 40 headers + 0 data bytes



--- doblog-uow.1ez.xyz hping statistic ---
1000 packets transmitted, 0 packets received, 100% packet loss
round-trip min/avg/max = 0.0/0.0/0.0 ms


CalledProcessError: Command 'b'sudo hping3 -c 1000 -S -p 80 doblog-uow.1ez.xyz --faster\n'' returned non-zero exit status 1.

## Third Attack ~ Application Attack
This attack would be reflected in the applicaiton layer Apache Logs.

In [4]:
%%bash
wpscan --update

_______________________________________________________________
         __          _______   _____
         \ \        / /  __ \ / ____|
          \ \  /\  / /| |__) | (___   ___  __ _ _ __ ®
           \ \/  \/ / |  ___/ \___ \ / __|/ _` | '_ \
            \  /\  /  | |     ____) | (__| (_| | | | |
             \/  \/   |_|    |_____/ \___|\__,_|_| |_|

         WordPress Security Scanner by the WPScan Team
                         Version 3.8.15
       Sponsored by Automattic - https://automattic.com/
       @_WPScan_, @ethicalhack3r, @erwan_lr, @firefart
_______________________________________________________________

[i] Updating the Database ...
[i] Update completed.



In [6]:
%%bash
wpscan --url http://doblog-uow.1ez.xyz/blog/

_______________________________________________________________
         __          _______   _____
         \ \        / /  __ \ / ____|
          \ \  /\  / /| |__) | (___   ___  __ _ _ __ ®
           \ \/  \/ / |  ___/ \___ \ / __|/ _` | '_ \
            \  /\  /  | |     ____) | (__| (_| | | | |
             \/  \/   |_|    |_____/ \___|\__,_|_| |_|

         WordPress Security Scanner by the WPScan Team
                         Version 3.8.15
       Sponsored by Automattic - https://automattic.com/
       @_WPScan_, @ethicalhack3r, @erwan_lr, @firefart
_______________________________________________________________

[+] URL: http://doblog-uow.1ez.xyz/blog/ [104.248.152.201]
[+] Started: Mon Mar  1 09:03:53 2021

Interesting Finding(s):

[+] Headers
 | Interesting Entry: Server: Apache/2.4.41 (Ubuntu)
 | Found By: Headers (Passive Detection)
 | Confidence: 100%

[+] XML-RPC seems to be enabled: http://doblog-uow.1ez.xyz/blog/xmlrpc.php
 | Found By: Direct Access (Aggressive Detec